In [1]:
import os

import numpy as np
import torch
from torch.utils.data import DataLoader
from lib.config import cfg
from lib.config import update_config

import lib.dataset.dataset3D as dataUtils
from lib.vortex.vortex import Vortex

In [2]:
training_params = {'batch_size': cfg.VORTEX.BATCH_SIZE,
                   'shuffle': True,
                   'drop_last': True,
                   'num_workers': cfg.DATALOADER_NUM_WORKERS,
                   'pin_memory': True}

val_params = {'batch_size': cfg.VORTEX.BATCH_SIZE,
              'shuffle': False,
              'drop_last': True,
              'num_workers': cfg.DATALOADER_NUM_WORKERS,
              'pin_memory': True}

training_set = dataUtils.VortexDataset3D(cfg, set='train')
val_set = dataUtils.VortexDataset3D(cfg, set='val')

training_generator = DataLoader(training_set, **training_params)
val_generator = DataLoader(val_set, **val_params)
calibPaths = [training_set.coco.dataset['calibration']['intrinsics'], 
              training_set.coco.dataset['calibration']['extrinsics']]

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [3]:
weights = '/home/trackingsetup/Documents/Vortex/projects/handPose_test/vortex/models/Colleen_d2_Run1/voxelnet_0_0.pth'

vortexNet = Vortex('train', cfg, calibPaths, '/home/timo/Desktop/VoRTEx/lib/vortex/lookup.npy', None)

/home/timo/Desktop/VoRTEx/lib/vortex/modules/v2vnet/repro_layer.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('boxsize_half',  torch.tensor(self.boxsize/2/self.grid_spacing).int())


[Info] initializing weights...


In [4]:
vortexNet.train(training_generator, val_generator, 1000)

  0%|          | 0/284 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import cv2
efficientDet = EfficientDet('inference', cfg, '/home/timo/Documents/Vortex/model_savefiles_combined/handPose_GN_2/efficientdet-d0_83_14112.pth')

out_dict = efficientDet.predict_on_image('/home/timo/Documents/CombiNet/datasets/handPose3D/train/Object_0/Camera_T/Frame_2096.jpg')
efficientDet.visualize_prediction(out_dict)

In [ ]:
efficientDet = EfficientDet('export', cfg, None)
efficientDet.export_to_onnx('test.onnx', input_size = 256)
